# Project Details
## Data wrangling, which consists of:

#### Gathering data

##### From 'twitter-archive-enhanced.csv' file.

##### From a link.

##### From twitter API.

#### Assessing data

#### Cleaning data

#### Storing, analyzing, and visualizing your wrangled data

#### Reporting on data wrangling efforts.

#### The data analyses and visualizations.

## Importing Libraries

In [ ]:
import pandas as pd
import numpy as np
import requests
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns; sns.set()
import os
import io
from twitter_api import get_twitter_data
import json
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import numpy as np
from PIL import Image


In [ ]:
#!pip install seaborn --upgrade #or also !pip install seaborn==0.9.0
import seaborn as sns
sns.__version__

## Gathering Data

In [ ]:
#load twitter archive file into pandas df.
df_arch = pd.read_csv('twitter-archive-enhanced.csv')

In [ ]:
#Download image_predictions file.
url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'
response = requests.get(url)
with open('image_predictions.tsv', mode ='wb') as file:
    file.write(response.content)

In [ ]:
#Reading TSV file
image_prediction = pd.read_csv('image_predictions.tsv', sep='\t' )

In [ ]:
#download data from twitter API 
if not os.path.exists('tweet_json.txt'):
    get_twitter_data(df_arch, 'tweet_json.txt')
    

In [ ]:
#load tweets data into pandas df
with open('tweet_json.txt') as file:
    df_api = pd.read_json(file, lines= True, encoding = 'utf-8')

In [ ]:
# Reduce df_api to the necessary columns
df_json = df_api[['id', 'retweet_count', 'favorite_count']]

In [ ]:
df_json.head(10)

---------------------------------------------------------------------------------------------------------

## Assessing

In [ ]:
df_arch

In [ ]:
df_arch.info()

In [ ]:
df_arch[['rating_numerator', 'rating_denominator']].describe()

In [ ]:
df_arch.name.unique()

In [ ]:
df_arch.name.value_counts()

In [ ]:
df_arch[df_arch.name.str.islower()]

In [ ]:
# looking for numerators with wrong values in chunks
df_arch[df_arch.rating_numerator <= 5].loc[0:1000, :]


In [ ]:
df_arch[df_arch.rating_numerator <= 5].loc[1000:2000, :]


In [ ]:
df_arch[df_arch.rating_numerator <= 5].loc[2000:, :]


In [ ]:
image_prediction

In [ ]:
image_prediction.info()

In [ ]:
image_prediction.columns

In [ ]:
image_prediction.img_num.value_counts()

In [ ]:
image_prediction.p1.unique()

In [ ]:
df_api

In [ ]:
df_api.info()

In [ ]:
df_api.lang.value_counts()

***

In [ ]:
df_arch.head(50)

In [ ]:
# Check if there are any records in df_arch are retweets
len(df_arch[df_arch.retweeted_status_id.isnull() == False])

In [ ]:
# Sort by rating_denominator values
df_arch.rating_denominator.value_counts().sort_index()

In [ ]:
# Sort by rating_numerator values
df_arch.rating_numerator.value_counts().sort_index()

In [ ]:
df_arch[['rating_numerator', 'rating_denominator']].describe()

In [ ]:
df_arch.name.unique()

## Observations
The following observations were made visually using Excel to view the data and Programmaticly using pandas functions.

### Quality

<font color=#0877cc size=4>**df_arch**</font>
1. Columns (`doggo`, `floofer`, `pupper`, `puppo`) has `None` for missing values.
2. `source` column is html tag `<a>` we can extract the source of the tweet and covert it to categorical.
3. `text` column has the link for the tweets and ratings at the end we can remove it.
4. `timestamp` column is `str` instead of `datetime`
5. We are interested in the tweet ONLY not the retweet there for we should remove those from the table.
6. We are interested in the tweet ONLY not the reply to the original tweet there for we should remove those from the table.
7. The `rating_numerator` column should of type `float` and also it should be correctly extracted.
8. `rating_denominator` column has values less than 10 and values more than 10 for ratings more than one dog.
9. `expanded_urls` column has NaN values
10. `id` column in df_api name different than the other 2 data sets.
11. `name` column have None instead of NaN and too many unvalid values.
12. ID variables are sometimes integers or floats (numeric)
13. "in_reply_to..." and "retweeted_status..." variables are numeric
14. retweets are present in the data
15. some of the column names are not  meaningful
16. "timestamp" and "retweeted_status_timestamp" are not a datetime variable
17. "source" values are formatted as <a> href=url </a>
18. rating_numerators are not always correctly accounting for decimals
19. the dog names are not standardized

## Structure (Tidiness)

<font color=#0877cc size=4>**df_arch**</font>
- `doggo`, `floofer`, `pupper`, `puppo` columns are all about the same things, a kind of dog personality.

<font color=#0877cc size=4>**image_prediction**</font>
- `img_num` useless.
- the columns (`p1`, `p1_dog`, `p1_conf`, ...etc) should be just `breed` and `confidence`.

<font color=#0877cc size=4>**df_api**</font>
- Just 3 columns needed `id`, `retweet_count`, `favorite_count`

<font color=#0877cc size=4>**In General**</font>
- All datasets should be combined into 1 dataset only

# Cleaning

### First let's take a copy from our data frames to work on cleaning it.

In [ ]:
df_arch_cleaned = df_arch.copy()
image_prediction_cleaned = image_prediction.copy()
df_api_cleaned = df_api.copy()

### Define
Replace 'None' with `np.nan` for Columns (`doggo`, `floofer`, `pupper`, `puppo`).

### Coding

In [ ]:
col_list = ['doggo', 'floofer', 'pupper', 'puppo']

for col in col_list:
    df_arch_cleaned[col] = df_arch_cleaned[col].replace('None', np.nan)

### Test

In [ ]:
df_arch_cleaned.info()

### Define
Extract tweet source from `source` column using `apply` meth in pandas and convert it to categorical.

### Coding

In [ ]:
# check for the unique values
df_arch_cleaned.source.unique()

In [ ]:
#make a function fix_source which extract the strings between tags
def fix_source(x):
    'x is an html string from the source column in df_arch_cleaned dataset'
    #find the first closed  tag >
    i= x.find('>') + 1
    # find the first open tag after the previous <
    j =x[i:].find('<')
    # extract the text in between
    return x[i:][:j]

In [ ]:
df_arch_cleaned.source = df_arch_cleaned.source.apply(lambda x: fix_source(x)).astype('category')

### Test

In [ ]:
# check for the unique values
df_arch_cleaned.source.unique()

### Define
Extract rating scores correctly from tweet text using RegEx and convert it to float

### Coding

In [ ]:
df_arch_cleaned[df_arch_cleaned.text.str.contains(r"(\d+\.\d*\/\d+)")][['text', 'rating_numerator']]

In [ ]:
new_ratings = df_arch_cleaned[df_arch_cleaned.text.str.contains(r"(\d+\.\d*\/\d+)")]['text'].str.extract(r"(\d+\.\d*(?=\/\d+))")
new_ratings

In [ ]:
df_arch_cleaned.loc[new_ratings.index, 'rating_numerator'] = new_ratings.values


In [ ]:
df_arch_cleaned.rating_numerator = df_arch_cleaned.rating_numerator.astype('float')

### Test

In [ ]:
df_arch_cleaned.loc[new_ratings.index]

In [ ]:
df_arch_cleaned.info()

### Define
Remove ratings and links from `text` column using `RegEx`.

### Coding

In [ ]:
df_arch_cleaned.text[0]

In [ ]:
df_arch_cleaned.text = df_arch_cleaned.text.str.extract('(.+(?=\s\d+/\d+\s))')

### Test

In [ ]:
df_arch_cleaned.text.sample(10)

### Define
Converte `timestamp` column to datetime.

### Coding

In [ ]:
df_arch_cleaned.timestamp = pd.to_datetime(df_arch_cleaned.timestamp)

### Test

In [ ]:
df_arch_cleaned.timestamp.dtype

### Define
Remove values other than 10 for `rating_denominator`

### Coding

In [ ]:
df_arch_cleaned = df_arch_cleaned[df_arch_cleaned['rating_denominator'] == 10]

### Test

In [ ]:
df_arch_cleaned[['rating_numerator', 'rating_denominator']].describe()

### Define
Remove any rows not related to dogs


### Coding

In [ ]:
df_arch_cleaned = df_arch_cleaned[~df_arch_cleaned.text.isnull()]

In [ ]:
df_arch_cleaned = df_arch_cleaned.loc[~df_arch_cleaned.text.str.match('.*only rate dogs')]

### Test

In [ ]:
df_arch_cleaned.loc[df_arch_cleaned.text.str.match('.*only rate dogs')]

In [ ]:
df_arch_cleaned.info()

### Define
Drop rows with NaNs for `expanded_urls` column.

### Coding

In [ ]:
df_arch_cleaned = df_arch_cleaned.loc[~df_arch_cleaned.expanded_urls.isnull()]

### Test

In [ ]:
df_arch_cleaned.info()

### Define
Create `dog_stage` column and remove the (`doggo`, `floofer`, `pupper`, `puppo`) columns.

In [ ]:
df_arch_cleaned

### Coding

In [ ]:
# select the dog stages columns from the dataset
cols = ['doggo', 'floofer', 'pupper', 'puppo']

# create the dog_stage column with joining the four columns in one column dog_stage join for more than stage
df_arch_cleaned['dog_stage'] = df_arch_cleaned[cols].\
                                        apply(lambda x: ', '.join(x.dropna().astype(str)),axis =1)
# replace the empty string with nan and change datatype to category
df_arch_cleaned.dog_stage = df_arch_cleaned.dog_stage.replace('', np.nan).astype('category')

# drop the 4 columns
df_arch_cleaned = df_arch_cleaned.drop(columns = cols, axis =1)

### Test

In [ ]:
df_arch_cleaned.info()

In [ ]:
df_arch_cleaned.dog_stage.value_counts()

### Define
- Replace 'None' with np.name in df_arch `name` column.
- Remove any rows with invalid names which starts with lower laters.

### Coding

In [ ]:
df_arch_cleaned[~df_arch_cleaned.name.str.istitle()].name.unique()

In [ ]:
df_arch_cleaned.name.replace(['such', 'a', 'quite', 'not', 'one', 'incredibly', 'mad',
       'an', 'very', 'just', 'my', 'his', 'actually', 'getting',
       'this', 'unacceptable', 'all', 'old', 'infuriating', 'the',
       'by', 'officially', 'life', 'light', 'space', 'None'], np.nan, inplace=True)

### Test

In [ ]:
df_arch_cleaned.name.unique()

In [ ]:
df_arch_cleaned.name.value_counts()

In [ ]:
df_arch_cleaned.info()

### Define
- Remove `img_num` column from image_prediction_cleand.

### Coding

In [ ]:
image_prediction_cleaned.drop('img_num', axis=1, inplace=True)

### Test

In [ ]:
image_prediction_cleaned.info()

### Define
Create `breed` and `confidence` columns with highest confidence predictions and drop other columns

### Coding

In [ ]:
breed = []
confidence = []
# iterating over image_prediction row by row and taking the highest confident prediction other wise np.nan
for index, row in image_prediction_cleaned.iterrows():
    if row['p1_dog'] and row['p1_conf'] == max([row['p1_conf'], row['p2_conf'], row['p3_conf']]):
        breed.append(row['p1'])
        confidence.append(row['p1_conf'])
    elif row['p2_dog'] and row['p2_conf'] == max([row['p1_conf'], row['p2_conf'], row['p3_conf']]):
        breed.append(row['p2'])
        confidence.append(row['p2_conf'])
    elif row['p3_dog'] and row['p3_conf'] == max([row['p1_conf'], row['p2_conf'], row['p3_conf']]):
        breed.append(row['p3'])
        confidence.append(row['p3_conf'])
    else:
        breed.append(np.nan)
        confidence.append(np.nan)
        
image_prediction_cleaned['breed'] = breed
image_prediction_cleaned['confidence'] = confidence


In [ ]:
image_prediction_cleaned = image_prediction_cleaned[['tweet_id', 'jpg_url', 'breed', 'confidence']]

### Test

In [ ]:
image_prediction_cleaned.sample(10)

In [ ]:
image_prediction_cleaned.info()

### Define
Remove unnecessary columns for df_api_cleand

### Coding

In [ ]:
df_api_cleaned.columns

In [ ]:
df_api_cleaned = df_api_cleaned[['id', 'retweet_count', 'favorite_count']]

### Test

In [ ]:
df_api_cleaned.info()

### Define
Rename `id` column in df_api_cleand to `tweet_id`

### Coding

In [ ]:
df_api_cleaned.columns = ['tweet_id', 'retweet_count', 'favorite_count']

### Test

In [ ]:
df_api_cleaned.columns

### Define
- merge data into database

### Coding

In [ ]:
df_merge = pd.merge(df_arch_cleaned, image_prediction_cleaned, on=['tweet_id'], how='inner')
df_merge = pd.merge(df_merge, df_api_cleaned, on = 'tweet_id', how = 'inner')


In [ ]:
df_merge.info()

In [ ]:
df_merge.head(10)

In [ ]:
# Calulate the value of 'rating'
df_merge['rating'] = df_merge['rating_numerator'] / df_merge['rating_denominator']

### STORING DATA

In [ ]:
df_merge.to_csv('archive_master_new.csv', encoding='utf-8',index=False)

### Test

In [ ]:
wrangled_df= pd.read_csv('archive_master_new.csv')

In [ ]:
wrangled_df.head(10)

***

In [ ]:
# this list of dogs for rating > 2
wrangled_df.loc[wrangled_df['rating'] > 2]

In [ ]:
data = wrangled_df.rating_numerator.value_counts()

x = data.index
y = data.values
fig, ax = plt.subplots(figsize=(12, 6))
g = sns.barplot(x, y, palette='Blues_d', ax=ax)
ax.set(xlabel='Ratings', ylabel='Frequency', title='Ratings frequency')
plt.show()

In [ ]:
data = wrangled_df.rating_numerator.value_counts()

ax = sns.boxplot(data, orient='v', width=.4)
ax.set(xlabel='Ratings', ylabel='Frequency', title='Ratings frequency')
plt.show()

As we can see that their are 2 outliers here so let's investigate more and check their data

In [ ]:
outliers_df = wrangled_df[wrangled_df.rating_numerator > 400][['rating_numerator', 'name', 'jpg_url', 'text']]
outliers_df

In [ ]:
if not os.path.exists('images'):
    os.makedirs('images')
fig=plt.figure()
c = 1
for index, row in outliers_df.iterrows():
    r = requests.get(row['jpg_url'])
    i = Image.open(BytesIO(r.content))
    i.save('image2/' +  str(index) + '_' + str(row['rating_numerator']) + "_" + str(row['name']) + '.jpg')
    fig.add_subplot(1, 2, c)
    c += 1
    plt.imshow(i)
    plt.axis("0")
plt.show()

### Define
- So we can see here that the picture with 1776 rating is simply very cute dog and has the best rating score but the other pic with 420 rating score is the rap star Snoop Dogg and should be removed from our data.

### Coding

In [ ]:
# Remove 'Snoop Dogg' from our wrangled data
wrangled_df.drop(index=1552, inplace=True)

### Test

In [ ]:
wrangled_df.query('rating_numerator == 420')

In [ ]:
wrangled_df.rating_numerator.unique()

- Next we can look for the relation betwee 'retweet_count' and 'favorite_count'

In [ ]:
import numpy as np
import pandas as pd
from os import path
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

import matplotlib.pyplot as plt
% matplotlib inline

In [ ]:
# Scatter plot to show the relation between favorits and retweets
ax = sns.regplot(x='retweet_count', y='favorite_count', data=wrangled_df)
plt.show()

In [ ]:
ax = sns.regplot(x='retweet_count', y='favorite_count', data=wrangled_df, color='b', scatter_kws={'s':5, 'alpha':.3}) 
ax.set(xlabel='Retweet count', ylabel='Favorite count', title='Favorits VS Retweets')
plt.show()

As we expected the favorite and retweets are highly postive correlated.

- Now let's compare dog stages

In [ ]:
data = wrangled_df.groupby('dog_stage').count()['tweet_id']
ax = sns.barplot(y=data.index, x=data.values, palette='Blues_d')
ax.set(xlabel='Count', ylabel='Dog stage', title='Dog Stage Counts')
plt.show()

As we can see the most common stage is pupper 

- Comparing tweets different sources

In [ ]:
data = wrangled_df.groupby('source').count()['tweet_id']
ax = sns.barplot(y=data.index, x=data.values, palette='Blues_d')
ax.set(xlabel='Count', ylabel='Tweet source', title='Tweet Source Counts')
plt.show()

And as we see here they tweeted the most from iPhone 

### Now let's create a funny world cloud from our tweets text

In [ ]:
text = " ".join(tweet for tweet in wrangled_df.text)
print ("There are {} words in the combination of all review.".format(len(text)))

In [ ]:
mask = np.array(Image.open('images/mask.jpeg'))
stopwords = set(STOPWORDS)
stopwords.update(['look', 'tho', 'see', 'good', 'hello', 'day', 'say', 'Meet'])
wordcloud = WordCloud( max_words=500, stopwords=stopwords, 
                      background_color='white', contour_color='black', mask=mask).generate(text)

# Display the generated image:
# the matplotlib way:
plt.figure(figsize=[20,10])
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.savefig('images/world_cloud.jpeg')
plt.show()